In [37]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve

In [38]:
ford_bs = pd.read_csv("C:/Users/Gast01/Desktop/Master Thesis Finance/Balance Sheets/Ford bs.csv",  on_bad_lines='skip')
ford_stock = pd.read_csv("C:/Users/Gast01/Desktop/Master Thesis Finance/Stock Prices/Ford Motor Stock Price History.csv")
ford_stock = ford_stock.iloc[::-1].reset_index(drop=True)

In [39]:
ford_melted = ford_bs.melt(id_vars=["Breakdown"], var_name="Date", value_name="Value")
required_items = ["Current Liabilities", "Long Term Debt", "Ordinary Shares Number"]
ford_filtered = ford_melted[ford_melted["Breakdown"].isin(required_items)]
ford_df = ford_filtered.pivot(index="Date", columns="Breakdown", values="Value").reset_index()
ford_df.columns.name = None  # Remove column grouping
ford_df.rename(columns={"Date": "Date"}, inplace=True)  # Keep Date explicit

In [40]:
ford_df["Date"] = pd.to_datetime(ford_df["Date"])
ford_stock["Date"] = pd.to_datetime(ford_stock["Date"])

ford_stock = ford_stock.sort_values(by="Date")

ford_df = pd.merge_asof(ford_df.sort_values("Date"), ford_stock[["Date", "Price"]], 
                        on="Date", direction="forward")

ford_df.rename(columns={"Price": "Stock Price"}, inplace=True)

In [41]:
ford_df["MktCap"] = ford_df["Stock Price"] * ford_df["Ordinary Shares Number"]
ford_df["Total Debt"] = ford_df["Current Liabilities"] + 0.2 * ford_df["Long Term Debt"]
ford_df["Asset Value"] = ford_df["MktCap"] + ford_df["Total Debt"]

In [42]:
ford_df = ford_df.iloc[::-1].reset_index(drop=True)

In [43]:
ford_df["Asset Return"] = np.log(ford_df["Asset Value"] / ford_df["Asset Value"].shift(1))
mu_A = ford_df["Asset Return"].mean()

In [44]:
ford_stock = ford_stock.iloc[::-1].reset_index(drop=True)
ford_stock["Date"] = pd.to_datetime(ford_stock["Date"])

# Set Date as index and create a full daily date range (including weekends)
ford_stock = ford_stock.set_index("Date").asfreq("D")  # "D" ensures daily frequency

# Forward fill missing values (fills Saturday and Sunday with Friday's values)
ford_stock = ford_stock.ffill()

ford_stock["Log Returns"] = np.log(ford_stock["Price"] / ford_stock["Price"].shift(1))

# Reset index
ford_stock = ford_stock.reset_index()

In [45]:
sigma_E = np.std(ford_stock["Log Returns"])

sigma_E

0.0198290115841388

In [46]:
D = ford_df["Total Debt"].iloc[-1]  # Latest total debt
E = ford_df["MktCap"].iloc[-1]  # Latest equity value
T = 1  # Time horizon in years
r = 0.03  # Risk-free rate (e.g., 3%)

def equations(vars):
    V, sigma_A = vars
    d1 = (np.log(V / D) + (r + 0.5 * sigma_A ** 2) * T) / (sigma_A * np.sqrt(T))
    d2 = d1 - sigma_A * np.sqrt(T)
    
    eq1 = V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E
    eq2 = sigma_E * E - sigma_A * V * norm.cdf(d1)
    
    return [eq1, eq2]


V_guess = E + D
sigma_A_guess = sigma_E * E / V_guess
V, sigma_A = fsolve(equations, (V_guess, sigma_A_guess))

In [47]:
d1 = (np.log(V / D) + (mu_A + 0.5 * sigma_A ** 2) * T) / (sigma_A * np.sqrt(T))
d2 = d1 - sigma_A * np.sqrt(T)
DD = d2  # Distance to Default

PD = norm.cdf(-DD)

print(f"The probability of default of Ford Motor Company is {PD:.4%}")

The probability of default of Ford Motor Company is 0.0000%
